函数

高阶函数（接受函数为参数，或者把函数作为结果返回的函数）

In [2]:
def factorial(n):
    return 1 if n in [0, 1] else factorial(n-2) + factorial(n-1)

list(map(factorial, filter(lambda x: x % 2, range(10))))
[factorial(i) for i in range(10) if i % 2]
# python2中 map和filter返回列表，所以最接近的替代品是列表推导
# python3中 map和filter返回生成器（一种迭代器），因此直接替代品是生成器表达式

from functools import reduce
from operator import add

reduce(add, range(10))
sum(range(10))
# reduce和sum的通用思想都是把某个操作作用到序列的元素上，累计之前的结果，把一系列值归约成一个值

# 其他的归约函数
all(range(10)) # all(iterable)
# 如果iterable的所有值都为真，返回True；如果empty, 如all([])，则返回True
any(range(10)) # any(iterable)
# 如果iterable中有元素为真，返回True；如果empty，如any([])，则返回False

True

匿名函数（不能赋值，也不能使用while/try等语句，只能使用纯表达式）

In [1]:
fruits = ['peach', 'apple', 'banana', 'mongo', 'cherry']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'peach', 'mongo', 'cherry']

可调用对象

调用运算符()可以运用到的对象，可以通过内置函数callable()函数来判断，有以下７种：
1. 用户定义的函数：用def/lambda表达式创建
2. 内置函数，如len
3. 内置方法，如dict.get
4. 方法：类中定义的函数
5. 类：会运行__new__方法创建一个实例，然后运行__init__方法初始化实例，最后把实例返回给调用方
6. 类的实例：需要定义__call__方法
7. 生成器函数：使用yield关键字的函数或方法，返回的是生成器对象

函数内省

In [4]:
class A: pass
a = A()
def f(): pass

sorted(set(dir(f)) - set(dir(a))) # 函数特有的属性

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

函数的参数

In [22]:
def f(a, b=1, *, c=2, d):
    return a, b, c, d

f(1, 2, c=3, d=4)
# python3提供了仅限关键字参数（keyword-only argument），如ｃ，仅限关键字参数不一定要有默认值
print(f.__defaults__) # 存放定位参数和关键字参数的默认值的元组
print(f.__kwdefaults__) # 存放仅限关键字参数的默认值（如果有的话）的字典

(1,)
{'c': 2}


In [30]:
from inspect import signature

def func(a, b=1, *c, d, **e):
    pass
sig = signature(func)
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)
# POSITIONAL_OR_KEYWORD 可以通过定位参数和关键字参数传入的形参
# VAR_POSITIONAL 定位参数元组
# VAR_KEYWORD 关键字参数字典
# KEYWORD_ONLY 仅限关键字参数，python3新增
# POSITIONAL_ONLY 并不支持

args = {
    'a': 1,
    'd': 3
}
bound_args = sig.bind(**args)
bound_args = sig.bind(1, d=3)
# bound_args = sig.bind(1, 3) # 会报错，因为d是仅限关键字参数
for name, value in bound_args.arguments.items():
    print(name, '=', value)
# inspect.Signature对象的bind方法可以把任意参数绑定到形参上，可以以此
# 在真正调用函数之前验证参数

POSITIONAL_OR_KEYWORD : a = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : b = 1
VAR_POSITIONAL : c = <class 'inspect._empty'>
KEYWORD_ONLY : d = <class 'inspect._empty'>
VAR_KEYWORD : e = <class 'inspect._empty'>
a = 1
d = 3


函数注解（TODO）

函数闭包

In [45]:
def a():
    v1 = 1
    def b():
        v1 = 2 # 此时创建了新的同名局部变量，不会对外部的v1造成影响
        # v1 = 1 # 此时在闭包函数ｂ中会默认v1为局部变量，所以如果有｀赋值｀操作则会报错
        # 除非使用nonlocal关键字声明v1，则可以外部的自由变量v1
    return b

# 闭包的三个条件
# 1. nested function 如上面的函数b
# 2. nested function引用enclosing function的变量，如b使用nonlocal v1的话
# 3. enclosing function必须返回nested function，如a返回了b

# 变量作用域
# LEGB: Local, Enclosing, Global, Built-in
# 关键字有nonlocal和global

# 闭包函数会保存定义enclosing function时存在的自由变量的绑定，
# 这样调用nested function虽然定义作用域不可用了，但是仍能使用那些绑定
def a():
    v1 = 1
    def b():
        nonlocal v1
        v1 = 2
    return b

b = a()
print(b.__closure__)
b.__closure__[0].cell_contents

(<cell at 0x7f5f842b9b58: int object at 0x55ca2bf8cf40>,)


1

装饰器

In [16]:
import functools


def dec1(func):
    @functools.wraps(func)
    def wrap(msg):
        print('start dec1')
        func(msg)
    return wrap

def dec2(func):
    @functools.wraps(func)
    def wrap(msg):
        print('start dec2')
        func(msg)
    return wrap

@dec1
@dec2
def print_msg(msg):
    print(msg)
    
print_msg('hello') # 装饰器会按照从上到下的顺序执行　即dec1(dec2(print_msg))('hello')
print(print_msg.__name__)
# 何时执行装饰器
registry = []
def reg(func):
    print('register func: %s', func)
    registry.append(func)
    
@reg
def func():
    print('running func')
    
print(registry) # 装饰器是在导入时或者说定义时就执行的，而不需要被装饰函数运行才会执行

# 标准库的装饰器
import time

def clock(func):
    @functools.wraps(func)
    def wrap(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_list = []
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, func.__name__, arg_str, result))
        return result
    return wrap

@clock
def factorial(n):
    return 1 if n in [0, 1] else factorial(n-2) + factorial(n-1)

# factorial(31)

@functools.lru_cache()
@clock
def factorial(n):
    return 1 if n in [0, 1] else factorial(n-2) + factorial(n-1)

# factorial(31) # 会缓存每次的结果，lru_cache带括号的原因是因为还支持别的参数

# 类装饰器
class log:
    def __init__(self, msg):
        self.msg = msg
        
    def __call__(self, func):
        @functools.wraps(func)
        def wrap(*args, **kwargs):
            print('logging: %s' % self.msg)
            return func(*args, **kwargs)
        return wrap
    
@log('test')
def f():
    print('running f')
    
f() # 类装饰器的好处是可以继承，例如可以加一个EmailLog(log)的类来增加发邮件的功能

start dec1
start dec2
hello
print_msg
register func: %s <function func at 0x7f4c5c81ea60>
[<function func at 0x7f4c5c81ea60>]
logging: test
running f
